In [ ]:
# I want to read in a hypnogram and gor contiguous groups of sleep sessions
# first I'll make a column of sleep or no sleep
# then I'll analyze the distribution of no sleep times
# hopefully there's a cutoff that I can run my pose estimation and heart rate analysis from

In [23]:
import pandas as pd

# let's start testing with the apple hypnogram 
workingDataPath = "/home/chowder/Documents/workingData/appleHK/sleep/"
dataFileName = "appleWatchHypnoDf.parquet.gzip"
hypnoDF = pd.read_parquet(workingDataPath + dataFileName).drop(["creationDate", "sourceVersion"], axis=1)
hypnoDF['startDate'] = hypnoDF.index
hypnoDF

,endDate,value,startDate
startDate,,,
2022-10-01 00:42:59-07:00,2022-10-01 01:13:59-07:00,1,2022-10-01 00:42:59-07:00
2022-10-01 01:13:59-07:00,2022-10-01 01:29:59-07:00,2,2022-10-01 01:13:59-07:00
2022-10-01 01:29:59-07:00,2022-10-01 01:30:29-07:00,0,2022-10-01 01:29:59-07:00
2022-10-01 01:30:29-07:00,2022-10-01 01:43:29-07:00,1,2022-10-01 01:30:29-07:00
2022-10-01 01:43:29-07:00,2022-10-01 01:50:59-07:00,3,2022-10-01 01:43:29-07:00
...,...,...,...
2024-07-16 07:14:28-07:00,2024-07-16 07:14:58-07:00,0,2024-07-16 07:14:28-07:00
2024-07-16 07:14:58-07:00,2024-07-16 07:34:58-07:00,1,2024-07-16 07:14:58-07:00
2024-07-16 07:34:58-07:00,2024-07-16 07:36:28-07:00,0,2024-07-16 07:34:58-07:00


In [34]:
hypnoDF["isSleep"] = hypnoDF["value"].map(lambda x:int(x > 0))
hypnoDF["isNewSleep"] = (hypnoDF["isSleep"] - hypnoDF["isSleep"].shift(1)).map(lambda x:int(x != 0))
hypnoDF['ConsecutiveGroup'] = hypnoDF['isNewSleep'].cumsum()
# there must be a simpler way to do this, but it works
# left join hypnodf
endTimesOfGroups = pd.merge(hypnoDF, 
                            hypnoDF.groupby('ConsecutiveGroup').endDate.agg(max), 
                            on='ConsecutiveGroup',
                            how='left')[['endDate_x', 'endDate_y']]

hypnoDF = pd.merge(hypnoDF, endTimesOfGroups, 
                   left_on='endDate', 
                   right_on='endDate_x').drop('endDate_x', axis=1)

startTimesOfGroups = pd.merge(hypnoDF, 
                               hypnoDF.groupby('ConsecutiveGroup').startDate.agg(min), 
                               on='ConsecutiveGroup',
                               how='left')[['startDate_x', 'startDate_y']]

hypnoDF = pd.merge(hypnoDF,startTimesOfGroups,
                   left_on='startDate', 
                   right_on='startDate_x').drop('startDate_x', axis=1)
hypnoDF

,endDate,value,startDate,isSleep,isNewSleep,ConsecutiveGroup,gEndDate,endDate_y,startDate_y
0,2022-10-01 01:13:59-07:00,1,2022-10-01 00:42:59-07:00,1,1,1,2022-10-01 01:29:59-07:00,2022-10-01 01:29:59-07:00,2022-10-01 00:42:59-07:00
1,2022-10-01 01:29:59-07:00,2,2022-10-01 01:13:59-07:00,1,0,1,2022-10-01 01:29:59-07:00,2022-10-01 01:29:59-07:00,2022-10-01 00:42:59-07:00
2,2022-10-01 01:30:29-07:00,0,2022-10-01 01:29:59-07:00,0,1,2,2022-10-01 01:30:29-07:00,2022-10-01 01:30:29-07:00,2022-10-01 01:29:59-07:00
3,2022-10-01 01:43:29-07:00,1,2022-10-01 01:30:29-07:00,1,1,3,2022-10-01 02:47:29-07:00,2022-10-01 02:47:29-07:00,2022-10-01 01:30:29-07:00
4,2022-10-01 01:50:59-07:00,3,2022-10-01 01:43:29-07:00,1,0,3,2022-10-01 02:47:29-07:00,2022-10-01 02:47:29-07:00,2022-10-01 01:30:29-07:00
...,...,...,...,...,...,...,...,...,...
17566,2024-07-16 07:14:58-07:00,0,2024-07-16 07:14:28-07:00,0,1,8286,2024-07-16 07:14:58-07:00,2024-07-16 07:14:58-07:00,2024-07-16 07:14:28-07:00
17567,2024-07-16 07:34:58-07:00,1,2024-07-16 07:14:58-07:00,1,1,8287,2024-07-16 07:34:58-07:00,2024-07-16 07:34:58-07:00,2024-07-16 07:14:58-07:00
17568,2024-07-16 07:36:28-07:00,0,2024-07-16 07:34:58-07:00,0,1,8288,2024-07-16 07:36:28-07:00,2024-07-16 07:36:28-07:00,2024-07-16 07:34:58-07:00
17569,2024-07-16 07:44:58-07:00,1,2024-07-16 07:36:28-07:00,1,1,8289,2024-07-16 07:44:58-07:00,2024-07-16 07:44:58-07:00,2024-07-16 07:36:28-07:00


In [ ]:
# rename Columns
hypnoDF['gEndDate'] = hypnoDF['endDate_y']
hypnoDF = hypnoDF.drop('endDate_y', axis=1)
hypnoDF['gStartDate'] = hypnoDF['startDate_y']
hypnoDF = hypnoDF.drop('startDate_y', axis=1)

In [43]:
# select and reorder columns
# desiredColumnOrder = ['startDate', 'gStartDate', 'endDate', 'gEndDate', 'ConsecutiveGroup', 'value']
# hypnoDF[desiredColumnOrder].head(20)

In [42]:
hypnoDF[['gStartDate', 'gEndDate']].drop_duplicates()

,gStartDate,gEndDate
0,2022-10-01 00:42:59-07:00,2022-10-01 01:29:59-07:00
2,2022-10-01 01:29:59-07:00,2022-10-01 01:30:29-07:00
3,2022-10-01 01:30:29-07:00,2022-10-01 02:47:29-07:00
10,2022-10-01 02:47:29-07:00,2022-10-01 02:48:59-07:00
11,2022-10-01 02:48:59-07:00,2022-10-01 07:27:29-07:00
...,...,...
17566,2024-07-16 07:14:28-07:00,2024-07-16 07:14:58-07:00
17567,2024-07-16 07:14:58-07:00,2024-07-16 07:34:58-07:00
17568,2024-07-16 07:34:58-07:00,2024-07-16 07:36:28-07:00
17569,2024-07-16 07:36:28-07:00,2024-07-16 07:44:58-07:00
